<a href="https://colab.research.google.com/github/BrainstormerAI/Use-Cases-of-Stable-Diffusion/blob/main/Backdrop%20Remix/Remove_or_Replace_background_using_SAM_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gradio==3.33.1
!pip install diffusers
!pip install accelerate
!pip install segment_anything

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached gradio-3.33.1-py3-none-any.whl (20.0 MB)
  Attempting uninstall: gradio
    Found existing installation: gradio 2.9.4
    Uninstalling gradio-2.9.4:
      Successfully uninstalled gradio-2.9.4
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip show gradio

Name: gradio
Version: 3.33.1
Summary: Python library for easily interacting with trained machine learning models
Home-page: 
Author: 
Author-email: Abubakar Abid <team@gradio.app>, Ali Abid <team@gradio.app>, Ali Abdalla <team@gradio.app>, Dawood Khan <team@gradio.app>, Ahsen Khaliq <team@gradio.app>, Pete Allen <team@gradio.app>, Ömer Faruk Özdemir <team@gradio.app>
License: 
Location: /usr/local/lib/python3.10/dist-packages
Requires: aiofiles, aiohttp, altair, fastapi, ffmpy, gradio-client, httpx, huggingface-hub, jinja2, markdown-it-py, markupsafe, matplotlib, mdit-py-plugins, numpy, orjson, pandas, pillow, pydantic, pydub, pygments, python-multipart, pyyaml, requests, semantic-version, typing-extensions, uvicorn, websockets
Required-by: 


In [ ]:
import gradio as gr
import numpy as np
import  torch
from diffusers import StableDiffusionInpaintPipeline
from PIL import Image
from segment_anything import SamPredictor, sam_model_registry


In [ ]:
device ="cuda"
sam_checkpoint ="/content/drive/MyDrive/weights for sam model for inpainting using stable diffusion/sam_vit_h_4b8939.pth"
model_type="vit_h"
sam=sam_model_registry[model_type](checkpoint=sam_checkpoint)
sam.to(device)
predictor=SamPredictor(sam)

In [ ]:
!pip install diffusers transformers accelerate scipy safetensors

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import torch
from diffusers import StableDiffusionInpaintPipeline
pipe = StableDiffusionInpaintPipeline.from_pretrained(
    "stabilityai/stable-diffusion-2-inpainting",
    torch_dtype=torch.float16,
)
pipe=pipe.to(device)


In [ ]:
selected_pixels=[]

In [ ]:
with gr.Blocks() as demo:
  with gr.Row():
    input_img = gr.Image(label="Input")
    mask_img = gr.Image(label="Mas")
    output_img = gr.Image(label="Output")

  with gr.Row():
    prompt_text=gr.Textbox(lines=1,label="Prompt")

  with gr.Row():
    submit = gr.Button("Submit")


  def generate_mask(image, evt: gr.SelectData):
    selected_pixels.append(evt.index)

    predictor.set_image(image)
    input_points=np.array(selected_pixels)
    input_labels=np.ones(input_points.shape[0])
    mask, _, _=predictor.predict(
        point_coords=input_points,
        point_labels=input_labels,
        multimask_output=False,
    )
    mask=np.logical_not(mask)
    mask = Image.fromarray(mask[0, :, :])
    return mask

  def inpaint(image,mask,prompt):
    image=Image.fromarray(image)
    mask=Image.fromarray(mask)

    image = image.resize((512,512))
    mask = mask.resize((512,512))

    output = pipe(
        prompt=prompt,
        image=image,
        mask_image=mask,
    ).images[0]

    return output

  input_img.select(generate_mask, [input_img],[mask_img])
  submit.click(
      inpaint,inputs=[input_img, mask_img, prompt_text],
      outputs=[output_img],
  )

if __name__ == "__main__":
  demo.launch()


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>